In [1]:
import os

In [2]:
%pwd


'e:\\projects\\TB_Chest_Radiography_Classification_Project\\research'

In [3]:
os.chdir("../ ")

update the entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestinConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

5. Update the configuration manager in src config

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestinConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestin_config = DataIngestinConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestin_config

6. Update the components

In [7]:
import os
import urllib.request as request
import shutil
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestinConfig):
        self.config = config

    def download_data(self):
            
        local_source = "artifacts/data_ingestion/TB_Chest_Radiography_Database"
        
        if os.path.exists(local_source):
            # Copy from local source
            if not os.path.exists(self.config.local_data_file):
                if os.path.isfile(local_source):
                    # If it's a single file
                    shutil.copy2(local_source, self.config.local_data_file)
                else:
                    # If it's a directory, copy the contents
                    os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
                    shutil.copytree(local_source, self.config.local_data_file)
                logger.info(f"File copied from local source: {local_source}")
            else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        else:
            # Fallback to original download method
            if not os.path.exists(self.config.local_data_file):
                filename, header = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"File : {filename} downloaded with following info: \n{header}")
            else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        

    def extract_zip_file(self):
        """
        Since your data is not a zip file, this method will verify the dataset structure
        Function returns None
        """
        # Create the unzip directory for consistency with the pipeline
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        # Verify the TB dataset structure
        if os.path.exists(self.config.local_data_file):
            normal_path = os.path.join(self.config.local_data_file, "normal")
            tuberculosis_path = os.path.join(self.config.local_data_file, "tuberculosis")
            
            if os.path.exists(normal_path) and os.path.exists(tuberculosis_path):
                normal_count = len(os.listdir(normal_path))
                tb_count = len(os.listdir(tuberculosis_path))
               
    

7. Update the pipeline

In [9]:
try:
    config  = ConfigurationManager()
    data_ingestin_config = config.get_data_ingestion_config()
    data_ingestin = DataIngestion(config = data_ingestin_config)
    data_ingestin.download_data()
    data_ingestin.extract_zip_file()
except Exception as e:
    raise e

[2025-09-07 07:59:14,235 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-09-07 07:59:14,248 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-09-07 07:59:14,250 : INFO : common : created directory at: artifacts]
[2025-09-07 07:59:14,251 : INFO : common : created directory at: artifacts/data_ingestion]
[2025-09-07 08:00:14,743 : INFO : 2358679216 : File copied from local source: artifacts/data_ingestion/TB_Chest_Radiography_Database]
